# Implementation of Finite Mixture Model

<details>
  <summary>Table of Contents</summary>
  <ol>
    <li>
      <a href="#import-needed-filepaths-and-libraries">Import Needed Filepaths and Libraries</a>
    </li>
    <li>
    <a href="#load-dataset-into-pandas-dataframe">
    Load Dataset Into Pandas DataFrame
    </a>
    </li>
    <li>
    <a href="#define-response-variable-y">
    Define Response Variable y
    </a>
    </li>
    <li>
    <a href="#set-up-reproducible-random-number-generator">
    Set Up Reproducible Random Number Generator
    </a>
    </li>
    <li>
      <a href="#setting-up-mixture-model">Setting Up Mixture Model</a>
      <ul>
        <li><a href="#initial-parameters">Initial Parameters</a></li>
        <li><a href="#setting-priors">Setting Priors</a></li>
      </ul>
    </li>
    <li><a href="#gibbs-sampler-implementation">Gibbs Sampler Implementation</a>
      <ul>
        <li><a href="#gibbs-sampler-updates">Gibbs Sampler Updates</a></li>
      </ul>
</li>
  </ol>
</details>

## Import Needed Filepaths and Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from texas_gerrymandering_hb4.config import FINAL_CSV

## Load Dataset Into Pandas DataFrame
Our processed dataset is read into a Pandas DataFrame.

In [ ]:
df = pd.read_csv(FINAL_CSV)

## Define Response Variable y
To clarify, `y` represents an array of outcomes.

In [ ]:
y = df["dem_share"].values.astype(float)

## Set Up Reproducible Random Number Generator

In [ ]:
np.random.seed(123)

## Setting Up Mixture Model

## Initial Parameters
* These parameters represent starting guesses for the Gibbs sampler.
* `lambda` ($\lambda$) is the mixing proportion. Starting from a place of ignorance, we assume a perfect mix between the two, and that our means are the sample means and our variances are the sample variances. $\lambda$ is set to 0.5 to serve as a neutral initial guess.
* `mu_1` and `mu_2` represent the initial means for each component.
* `sigma_squared_1` and `sigma_squared_2` are the initial variances for each component.

In [ ]:
lambda = 0.5
mu_1 = np.mean(y)
mu_2 = np.mean(y)
sigma_squared_1 = np.var(y)
sigma_squared_2 = np.var(y)

### Setting Priors
* `alpha_1` and `alpha_2` are the priors for lambda.
$$\lambda \sim Beta(\alpha_2, \alpha_2)$$
Because the prior for lambda is Beta(2,2), `alpha_1` and `alpha_2` are both set to 2.
* This is a conjugate prior. Also, as a reminder, Beta(2,2) is also a Dirchilet distribution. What is beneficial about this is that the probability of obtaining 0 or 1, which is a degenerate model, is 0. Hence, as you get closer to 0 or 1, the likelihood is tiny. As a result, the problem is pushed further away from a degenerate value, so they become less likely to accidently become a point of convergence.
* The means `m0_1` and `mu0_2` are coming from the same distribution.
* Our variances are coming from the scaled inverse-$\chi^2$ distribution with 2 degrees of freedom.

In [ ]:
alpha_1 = 2
alpha_2 = 2

mu0_1 = np.mean(y)
mu0_2 = np.mean(y)

sigma_0_squared_1 = np.var(y)
sigma_0_squared_2 = np.var(y)

### Gibbs Sampler Parameters

In [ ]:
iterations = 1000
warmup = 500

## Storage for Samples

In [ ]:
lambda samples = np.zeros(iterations)
mu1_samples = np.zeros(iterations)
mu2_samples = np.zeros(iterations)
sigma_squared_1_samples = np.zeros(iterations)
sigma_squared_2_samples = np.zeros(iterations)

## Scaled Inverse-$\chi^2$ Sampler

## Gibbs Sampler Implementation
* The process for the Gibbs sampler involves initializing parameters, iterative sampling, and continuing the iterations until convergence.
* The benefits of the Gibbs sampler are that it handles complex, high-dimensional distributions and is easier to implement than direct sampling methods.

###  Gibbs Sampler Updates

#### Updating z
* We compute the posterior probability that our $z_{i}=1$, taking into account the normal distributions that our data follows:
$$p_{z_{i}=1} = \frac{\lambda_{old} \cdot N(y_{i}|\mu_{1,2}, \sigma^2_{1,2})}{\lambda_{old} \cdot N(y_{i}|\mu_{1,2}, \sigma_{1,2}^2) + (1-\lambda_{old})N(y_{i}|\mu_{1,2}, \sigma_{1,2}^2)}$$
* Once the probability of $z_{i=1}$ is computed, then a new value of $z_{i}$ is computed using a binomial distribution.
$$z_{i}^{new} \leftarrow Bin(n, p_{z_{i}=1})$$

#### Updating $\lambda$
* Updating $\lambda$ is a two-step process because it follows a Beta distribution.
* First, we update the Beta distribution using the standard form from conjugacy. Recall that our parameters are (2,2); they are being updated based on the frequency of 1's among our z's, which is why we end up with two updates for $\alpha$ and $\beta$. We begin updating $\lambda$ by first updating $\alpha$ and $\beta$ parameters:
$$\alpha_{new} \leftarrow \alpha_{old} + \sum z_{i}^{new}$$
$$\beta_{new} \leftarrow \beta_{old} + n -  \sum z_{i}^{new}$$
* We update $\lambda$ by computing the posterior probability. Because our $\lambda$ is coming from a Beta distribution with updated parameters, we draw a value of $\lambda$:
$$\lambda_{new} \leftarrow Beta(\alpha_{new}, \beta_{new})$$

#### Updating Means
* Our number of observations are $n_1 = \sum z_{i}^{new}$ and $n_2 = \sum z_{i}^{new}$, respectively. Recall that our $z_{i}$ values have been redrawn.
* We examine the total number of observations, followed by the means.  We compute the means as follows:
$$\bar{y}_1 = \frac{1}{n_1} \sum y_{i} (z_{i}^{new} = 1)$$
$$\bar{y}_2 = \frac{1}{n_2} \sum y_{i} (z_{i}^{new} = 1)$$
* Once we obtain our means, we are able to compute a posterior mean based our sample. We construct this using our initial values; we use `init` to denote this. This is combined with our prior and sample to compute a new mean as follows:
$$\mu_{1,new} = \frac{\frac{\mu}{\sigma_{1,init}^2} + n_1 \frac{\bar{y}_1}{\sigma^2_{1, old}}}{\frac{1}{\sigma_{1,init}^2}}$$